# Dataprocessing the Pecan dataset using pandas

### Importing libraries

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime
plt.style.use('ggplot')
import matplotlib.dates as dates
import warnings
warnings.filterwarnings("ignore")

# plots directory
figure_directory= 'figures/'
# change so that all figures have font size defulat as 22
plt.rcParams.update({'font.size': 18})
# figure size always the same
plt.figure(figsize=(16,12))

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

In [2]:
import os
import sys
import pandas as pd
from collections import defaultdict
import glob

In [4]:
def read_data():
    df = pd.DataFrame()
    houses = []
    path = "datasets/*.csv"
    for fname in glob.glob(path):
        house_df = pd.read_csv(fname)
        houseID = int(fname.split('.')[0].split('\\')[1])
        
        house_df['DateTime'] = pd.to_datetime(house_df['DateTime'])
        mask = (house_df['DateTime'] >= '2014-1-1') & (house_df['DateTime'] < '2015-1-1')
        house_df = house_df.loc[mask]
        house_df = house_df.set_index('DateTime')
        house_df = house_df.resample('1H').first()  # resample from 15 mins to 1 hour 
        house_df = house_df.reset_index(drop=False)
        
        if len(house_df) == 8760 and house_df.isnull().sum().sum() <= 100: # one year hours 
            houses.append(houseID)
            house_df.columns = ['localhour', 'use', 'air1', 'furnace1', 'dishwasher1', 'regrigerator1']  
            house_df = house_df.fillna(method='pad') 
            house_df['regrigerator1'] = house_df['regrigerator1'].fillna(method='ffill')
            appliances_sum = house_df[['air1', 'furnace1', 'dishwasher1', 'regrigerator1']].sum(axis=1)
            house_df['other'] = house_df['use'].subtract(appliances_sum)
            house_df['house'] = houseID
            house_df = house_df.set_index('house')
           
            df = pd.concat([df, house_df])
                               
    return df, houses

df, houses = read_data()
df.head()

,localhour,use,air1,furnace1,dishwasher1,regrigerator1,other
house,,,,,,,
135,2014-01-01 00:00:00,1.043,0.840,0.064,0.0,0.0,0.139
135,2014-01-01 01:00:00,0.158,0.000,0.009,0.0,0.0,0.149
135,2014-01-01 02:00:00,1.097,0.759,0.060,0.0,0.0,0.278
135,2014-01-01 03:00:00,0.715,0.522,0.045,0.0,0.0,0.148
135,2014-01-01 04:00:00,0.111,0.000,0.009,0.0,0.0,0.102


In [ ]:
df.info()

In [ ]:
df.describe()

In [14]:
def format_data(df, houses):
    '''
    Parameters: dataframe of the apppliacnes
    Return: dictionary contains all X^T x m
    '''
    d = {}
    for appliance in df.columns.tolist():
        started = 0
        
        for i in houses:
            if started == 0:
                d[str(appliance)] = df[[str(appliance)]][df[str(appliance)].index == i]
                started = 1
                dfindex = d[str(appliance)].index    
            else:
                d[str(appliance)][str(i)] = pd.Series(df[str(appliance)][df[str(appliance)].index == i].values,index=dfindex)

        d[str(appliance)]=d[str(appliance)].rename(columns = {str(appliance):str(dfindex[0])})
        d[str(appliance)].reset_index(drop=True, inplace=True)
        
    return d
    
dic = format_data(df, houses)
dic['use'].tail(5)

,135,2004,2034,2156,2171,2242,2710,275,2829,2845,...,3273,3443,3678,3687,3721,3873,3938,508,898,94
8755,0.827,0.416,1.346,6.396,1.188,6.182,3.999,0.793,2.192,1.302,...,0.358,1.688,1.579,0.247,5.551,0.476,0.190,0.522,0.807,2.251
8756,0.371,0.353,1.629,3.245,1.648,5.136,3.027,1.556,2.470,1.134,...,3.503,0.381,1.199,0.697,3.926,2.414,0.450,0.779,0.360,1.235
8757,0.596,0.453,0.532,3.343,0.761,4.054,14.056,1.399,2.345,0.919,...,3.265,0.381,1.120,0.763,0.424,0.100,0.151,1.770,0.242,1.081
8758,1.096,0.355,0.318,1.780,0.872,2.633,6.247,1.413,2.432,0.654,...,2.557,1.325,0.979,0.935,0.497,0.209,0.452,1.029,0.244,1.115
8759,0.522,0.759,0.399,1.414,1.903,1.836,6.971,1.093,3.066,1.051,...,0.918,0.264,0.838,1.889,1.259,0.100,0.474,0.879,0.336,1.079


In [ ]:
key = list(dic.keys())
key
# columns = dic[key].columns.tolist()

In [23]:
def split(d,portion,timeframe):
    '''
    Parameters: d = dictionary, portion 0.5 - 0.9, timeframe 1-8760

    Return: x_train,x_test dictionarys containing dataframes of all the appliances within the timeframe.
    '''
    x_train = {}
    x_test = {}
    timeframe = range(timeframe)
    train_list  = timeframe[int(len(timeframe) * 0.0):int(len(timeframe) * portion)]
    test_list = timeframe[int(len(timeframe) * portion):int(len(timeframe) * 1.0)]

    for key in d.keys():
        x_train[key] = d[key].loc[train_list,:]
        x_test[key] = d[key].loc[test_list,:]

    return x_train,x_test

### Splitting the dataframes into portions of a month

In [24]:
t = 12

x_train,x_test = split(dic, 0.5, t)
x_train_use = x_train.pop('use',None)
x_test_use = x_test.pop('use',None)
x_train_localhour = x_train.pop('localhour',None)
x_test_localhour = x_test.pop('localhour',None)

In [ ]:
x_train[list(x_train.keys())[0]].shape

In [ ]:
x_train.values()

## Splitting into weekdays and weekends

In [ ]:
week = pd.DatetimeIndex(pd.to_datetime(df['localhour'])).weekday
weekdays = np.in1d(week.ravel(),range(5)).reshape(week.shape)
weekends = week[np.where(week > 4,week,0)]
weekends = np.in1d(week.ravel(),[5,6]).reshape(week.shape)

In [ ]:
df[weekdays].describe()

In [ ]:
df[weekends].to_csv('weekendpecanhour2014')
df[weekdays].to_csv('weekdayspecanhour2014')

In [ ]:
df[weekends].describe()

### Data Exploration

In [ ]:
local = df['localhour'].tolist()
import datetime, re
xticks = map(lambda x: re.split('\s',x)[0],local)

### Usage for texas is like a poisson distribution, could be used in future research

In [ ]:
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy import stats
plt.figure(figsize=(16,8))

#df_plot = np.where(df['use'].values < 6)
df_plot = df['use'].values[np.where(df['use'].values < 6)]
#entries, bin_edges, patches = df_plot[df_plot < 6].hist(bins=1000, figsize=(16,12),normed=True)
# get poisson deviated random numbers
data = np.random.poisson(2, 1000)
entries, bin_edges, patches = plt.hist(df_plot,bins=1000,normed=True,label='Household energy usage', alpha=0.4)

# the bins should be of integer width, because poisson is an integer distribution
#entries, bin_edges, patches = plt.hist(data, bins=11, range=[-0.5, 10.5], normed=True)

# calculate binmiddles
bin_middles = 0.5*(bin_edges[1:] + bin_edges[:-1])

# poisson function, parameter lamb is the fit parameter
def poisson(k, lamb):
    return (lamb**k/factorial(k)) * np.exp(-lamb)

def weib(x,n,a):
    return (a / n) * (x / n)**(a - 1) * np.exp(-(x / n)**a)

(loc, scale) = stats.exponweib.fit_loc_scale(data, 1, 1)
print(loc, scale)

# fit with curve_fit
parameters, cov_matrix = curve_fit(poisson, bin_middles, entries) 

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 6, 1000)

# plt.plot(x, weib(x, loc, scale))
# plt.hist(data, data.max(), normed=True)
curve = plt.plot(x_plot, poisson(x_plot, *parameters), 'r-', lw=2,label='Fitted poisson distribution')

plt.title('Histogram of the whole-home usage for the Pecan dataset')
plt.xlabel('kW')
plt.legend()

# plt.savefig(figure_directory+'histusage.png')

In [ ]:
from scipy import stats
plt.figure(figsize=(16,8))
x_plot = np.linspace(0, 6, 1000)
#df_plot = np.where(df['use'].values < 6)
df_plot = df['use'].values[np.where(df['use'].values < 6)]
plt.plot(x_plot, stats.exponweib.pdf(x_plot, *stats.exponweib.fit(df_plot, 1, 1, scale=0.2, loc=0)),lw=2,label='Fitted weibull distribution')
_ = plt.hist(df_plot, bins=1000, normed=True, alpha=0.5, label='Household energy usage');

plt.title('Histogram of the whole-home usage for the Pecan dataset')
plt.xlabel('kW')
plt.legend()

# plt.savefig(figure_directory+'new_histusage.png')

In [ ]:
df_test = df.copy()
df_test.drop('localhour', axis=1, inplace=True)
hours = 24*7
df_test[0:hours].reset_index(drop=True).plot(figsize=(16,8))
plt.title('A week of daily consumption for a household')
plt.xlabel('Hour')
plt.ylabel('kW')

In [ ]:
import numpy.ma as ma

df_sort =df.set_index(df['localhour']).sort_index(by='localhour')
average_use = list(map(lambda x: df_sort.loc[x,'use'].mean(),df_sort.index.unique()))
leng = len(average_use)

mean_use = np.mean(np.where(np.isnan(average_use), ma.array(average_use, mask=np.isnan(average_use)).mean(axis=0), average_use))

fig, ax = plt.subplots(figsize=(16,8))
houses = df.index.unique()
i = 0
for house in houses:
    df.loc[house,'use'].reset_index(drop=True).plot(color='DarkBlue', alpha=0.6, label='Household energy' if i == 0 else '', ax=ax)
    i += 1

s = pd.Series(average_use)

s_df = pd.DataFrame({'Average':s})
#df_use.set_index()

s_df.plot(color='green',label='Average Use',lw=1, ax=ax)

plt.title('A year of consumption data for all {} households in training set'.format(len(houses)))
plt.xlabel('Hours')
plt.ylabel('kW')
from collections import OrderedDict
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
at = AnchoredText("Average consumption: {:.4f} kw".format(mean_use),
                  prop=dict(size=20), frameon=True,
                  loc=2,
                  )

at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax.add_artist(at)

### plotting the weekend household use of the week dataset

In [ ]:
df = pd.read_csv('weekendpecanhour2014')
df = df.set_index(df['house'])
df_sort =df.set_index(df['localhour']).sort_index(by='localhour')
average_use = list(map(lambda x: df_sort.loc[x,'use'].mean(),df_sort.index.unique()))
leng = len(average_use)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
houses = df.index.unique()
i = 0
for house in houses:
    df.loc[house,'use'].reset_index(drop=True).plot(color='DarkBlue', alpha=0.6, label='Household energy' if i == 0 else '', ax=ax)
    i += 1

mean_use = np.mean(np.where(np.isnan(average_use), ma.array(average_use, mask=np.isnan(average_use)).mean(axis=0), average_use))
    
s = pd.Series(average_use)
s_df = pd.DataFrame({'Average':s})
#df_use.set_index()

s_df.plot(color='Green',label='Average Use',lw=1, ax=ax)

plt.title('The weekend consumption data for all {} households in training set'.format(len(houses)))
plt.xlabel('Hours')
plt.ylabel('kW')
from collections import OrderedDict
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
at = AnchoredText("Average consumption: {:.4f} kW".format(mean_use),
                  prop=dict(size=20), frameon=True,
                  loc=2,
                  )
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax.add_artist(at)

In [ ]:
df = pd.read_csv('weekdayspecanhour2014')
df = df.set_index(df['house'])
df_sort =df.set_index(df['localhour']).sort_index(by='localhour')
average_use = list(map(lambda x: df_sort.loc[x,'use'].mean(),df_sort.index.unique()))
leng = len(average_use)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
houses = df.index.unique()
i = 0
for house in houses:
    df.loc[house,'use'].reset_index(drop=True).plot(color='DarkBlue', alpha=0.6, label='Household energy' if i == 0 else '', ax=ax)
    i += 1

mean_use = np.mean(np.where(np.isnan(average_use), ma.array(average_use, mask=np.isnan(average_use)).mean(axis=0), average_use))
    
s = pd.Series(average_use)
s_df = pd.DataFrame({'Average':s})
#df_use.set_index()

s_df.plot(color='Green',label='Average Use',lw=1, ax=ax)

plt.title('The weekdays consumption data for all {} households in training set'.format(len(houses)))
plt.xlabel('Hours')
plt.ylabel('kW')
from collections import OrderedDict
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
at = AnchoredText("Average consumption: {:.4f} kW".format(mean_use),
                  prop=dict(size=20), frameon=True,
                  loc=2,
                  )
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax.add_artist(at)